In [ ]:


sentence_transformer_model_list = ['all-mpnet-base-v2','all-distilroberta-v1','all-MiniLM-L6-v2','multi-qa-distilbert-cos-v1','paraphrase-MiniLM-L6-v2','distiluse-base-multilingual-cased-v1','distiluse-base-multilingual-cased-v2','paraphrase-multilingual-MiniLM-L12-v2','msmarco-distilbert-cos-v5','multi-qa-mpnet-base-cos-v1']
sentence_transformer_model_list+=['thenlper/gte-large',
 'thenlper/gte-base',
 'intfloat/e5-base-v2',
 'embaas/sentence-transformers-e5-large-v2',
 'jinaai/jina-embeddings-v2-base-en']


def embed_content(text_list, model='gemini', similarity_type='semantic_similarity', device=device):
    print('model for embedding', model)
    print(len(text_list))
    if model=='gemini':
        genai.configure(api_key=paid_key_gemini)
        model_name="models/text-embedding-004"
        task_type = similarity_type
        embedded_content = genai.embed_content(
        model= model_name,
        content=text_list,
        task_type=task_type,
        )['embedding']

        return  embedded_content

    elif model=='cohere':

        embedded_content = cohere_client.embed(
        model='embed-english-v3.0',
        texts=text_list,
        input_type='search_query',
        truncate='END'
        )
        return embedded_content.embeddings

    elif model =='text-embedding-3-small' or model =='text-embedding-3-large':
        print('model ', model)
        client = OpenAI(api_key=openai_api_key, organization ='org-PvvGd5GqUSJXbHLdTVVN3nzr')

        batch_size = 128
        embeddings = []
        for i in range(0, len(text_list), batch_size):
            print('i', i)
            batch = text_list[i:i + batch_size]
            response = client.embeddings.create(
                model=model, input=batch, encoding_format="float"
            )
            batch_embeddings = [x.embedding for x in response.data]
            embeddings.extend(batch_embeddings)

        embeddings = np.array(embeddings)
        return embeddings



    elif model=='voyage-3-lite':

        batch_size = 128
        embeddings = []
        for i in range(0, len(text_list), batch_size):
            print('i', i)
            batch = text_list[i:i + batch_size]
            voyageai.api_key = voyage_api_key
            vo = voyageai.Client()
            batch_embeddings = vo.embed(batch, model="voyage-3-lite", input_type="document").embeddings
            embeddings.extend(batch_embeddings)
            # time.sleep(1)
        return embeddings

    elif model=='Mistral':

        batch_size = 100
        all_embeddings = []

        print('batch_size ', batch_size)

        for i in range(0, len(text_list), batch_size):
            print('i',i)
            print('sleep')
            time.sleep(2)


            batch = text_list[i:i+batch_size]
          # In a real scenario, you would call the API here for the batch
          # Example with a dummy return:
          # if model == 'Mistral':
            # Simulate a call to Mistral API
            # import os
            # from mistralai.client import MistralClient
            # from mistralai.models.embeddings import EmbeddingRequest

            # mistral_key = os.environ.get("MISTRAL_API_KEY") # Make sure your API key is in environment variable
            # if not mistral_key:
                # raise ValueError("MISTRAL_API_KEY environment variable not set")

            client = Mistral(api_key=mistral_key)

            # print(f"Embedding batch {i//batch_size + 1}/{len(text_list)//batch_size + 1}")
            # print(f"Batch size: {len(batch)}")

            try:
                embeddings_batch_response = client.embeddings.create(
                    model="mistral-embed", # Use the correct Mistral embedding model name
                    inputs=batch,
                    # The API might have a different parameter name for inputs,
                    # referring to the documentation is crucial. It seems 'inputs' was
                    # used in the traceback, but 'input' is common in newer clients.
                    # Let's stick to 'input' based on potential SDK updates or common practice.
                )
                batch_embeddings = [data.embedding for data in embeddings_batch_response.data]
                all_embeddings.extend(batch_embeddings)
            except Exception as e:
                print(f"Error embedding batch starting at index {i}: {e}")
                # Depending on your needs, you might want to handle errors differently,
                # like retrying or logging the specific texts that failed.
                # For now, we'll raise the exception to stop execution if an error occurs.
                raise




        # time.sleep(10)


        # model = "mistral-embed"
        # print('text_list ', len(text_list))

        # client = Mistral(api_key=mistral_key)

        # embeddings_batch_response = client.embeddings.create(
        #     model=model,
        #     inputs=text_list,
        # )

        # print('embeddings_batch_response',embeddings_batch_response)


        # all_embeddings_list = []

        # # Iterate through the data in the response and extract each embedding vector
        # if embeddings_batch_response.data:
        #     for x in embeddings_batch_response.data:
        #     # for embedding_data_object in embeddings_batch_response.data:
        #       all_embeddings_list.append(x.embedding)



        # batch_size = 128
        # embeddings = []
        # for i in range(0, len(text_list), batch_size):
        #     print('i', i)
        #     batch = text_list[i:i + batch_size]
        #     voyageai.api_key = voyake_api_key
        #     vo = voyageai.Client()
        #     batch_embeddings = vo.embed(batch, model="voyage-3-lite", input_type="document").embeddings
        #     embeddings.extend(batch_embeddings)
        #     # time.sleep(1)
        return all_embeddings# embeddings_batch_response



    elif model in sentence_transformer_model_list:
        print('sentence transformer ', model)
        if model in sentence_transformer_models_dict:
            model_embed = sentence_transformer_models_dict[model]
        else:
            print('create model object ' , model)
            model_embed = SentenceTransformer(model, device=device)
            sentence_transformer_models_dict[model]=model_embed

        embeddings = model_embed.encode(text_list)
        return embeddings


In [ ]:
#



# safety_settings_gemini=[
# { "category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE" },
# { "category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE" },
# { "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE" },
# { "category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}
# ]






def content_replacement_of_text(content, max_tokens=5000, delay=0, model='sonnet'):

    print('model for deloc ', model)

    if len(content)< 10:
        print('No content ', content)
        return "", ""


    time.sleep(delay)


    prompt_prefix = f'Given below text, please convert all Person names(which are Proper Nouns) to a UNIQUE ID such as CHAR_A, CHAR_B, CHAR_C etc..  Keep it unique and for each UNIQUE Person name(which is a Proper Noun) use a UNIQUE ID. DO NOT KEEP THE ORIGINAL Person Names(which are Proper Nouns) in the generated paragraph text. Next, Replace all occurences  City/Country/Village/Town/River/Continent etc. names which are PROPER NOUNS to a UNIQUE ID such as LOC_A, LOC_B, LOC_C, LOC_D etc.. Next, Replace all occurences of company/organization names which are PROPER NOUNS to a UNIQUE ID such as ORG_A, ORG_B, ORG_C, ORG_D etc..   Do not replace monument/landmark names like Eiffel tower etc. Output contains the modified text only....\n  The text is provided below ::::\n\n '


    prompt = prompt_prefix + content

    prompt+='\n\n DO NOT GENERATE ANYTHING ELSE BEFORE OR AFTER. The output is of the format.\ n **MODIFIED_TEXT:** '


    # Format the request payload using the model's native structure.

    if model=='sonnet':
        native_request = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": int(max_tokens),
            "temperature": 0.9,
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": prompt}],
                }
            ],
        }

        # Convert the native request to JSON.
        request = json.dumps(native_request)

        config = Config(read_timeout=1000)

        brt = boto3.client(service_name=aws_service_name_aws,
        region_name=aws_region_name,
        config=config)

        try:
            # Invoke the model with the request.
            response = brt.invoke_model(modelId=aws_model_id, body=request)

        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{aws_model_id}'. Reason: {e}")
            return "", "ERROR " +str(e)

        # Decode the response body.
        model_response = json.loads(response["body"].read())

        # Extract and print the response text.
        response_text = model_response["content"][0]["text"]
        return response_text#,model_response

    elif model =='gemini':

        genai.configure(api_key=paid_key_gemini)
        gemini_model = "gemini-1.5-pro"


        model_object_gemini = genai.GenerativeModel(gemini_model)

        response_for_prompt = model_object_gemini.generate_content(prompt.encode().decode('unicode_escape')

                                                        , safety_settings = safety_settings_gemini

        )

        # print('response_for_prompt ', response_for_prompt)

        try:

            if len(response_for_prompt.candidates) >0:
                return response_for_prompt.text
        except:
            print('exception ')
            return ""




def content_remove_text(content, max_tokens=5000, delay=0, model='sonnet'):

    print('model for deloc ', model)

    if len(content)< 10:
        print('No content ', content)
        return "", ""

    # print('content ', content)

    time.sleep(delay)

# person names
# and location(cities/towns/countries etc. )
    prompt_prefix = f'Given below text, please COMPLETELY DELETE all Person/Character names which are PROPER NOUNS and City/Country/Village/Town/River/Continent/Organization etc. names which are PROPER NOUNS etc. Wherever they occur replace with empty string. Completely remove them and not anything else. Do not delete monument/landmark names like Eiffel tower etc. Do not remove He/She/him/her etc.. Output contains the modified text only....\n  The text is provided below ::::\n\n '

    prompt = prompt_prefix + content

    prompt+='\n\n DO NOT GENERATE ANYTHING ELSE BEFORE OR AFTER. The output is of the format.\ n **MODIFIED_TEXT:** '



    if model=='sonnet':
        native_request = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": int(max_tokens),
            "temperature": 0.9,
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": prompt}],
                }
            ],
        }

        # Convert the native request to JSON.
        request = json.dumps(native_request)

        config = Config(read_timeout=1000)

        brt = boto3.client(service_name=aws_service_name_aws,
        region_name=aws_region_name,
        config=config)

        try:
            # Invoke the model with the request.
            response = brt.invoke_model(modelId=aws_model_id, body=request)

        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{aws_model_id}'. Reason: {e}")
            return "", "ERROR " +str(e)

        model_response = json.loads(response["body"].read())

        response_text = model_response["content"][0]["text"]
        return response_text

    elif model =='gemini':

        genai.configure(api_key=paid_key_gemini)
        gemini_model = "gemini-1.5-pro"
        gemini_model = 'gemini-2.0-flash'

        model_object_gemini = genai.GenerativeModel(gemini_model)

        print(' len text ', len(prompt))

        response_for_prompt = model_object_gemini.generate_content(prompt.encode().decode('unicode_escape')

                                                        , safety_settings = safety_settings_gemini

        )


        try:

            if len(response_for_prompt.candidates) >0:
                return response_for_prompt.text
        except:
            print('exception ')
            RaiseError('exception in Gemini Anonymization')
            return ""


def text_anon(row, model, delay=0, REPLACEMENT_OR_REMOVAL=None):

    print('REPLACEMENT_OR_REMOVAL ', REPLACEMENT_OR_REMOVAL)

    if REPLACEMENT_OR_REMOVAL =='REPLACEMENT':
        modified_text = content_replacement_of_text(content=row['text'], model=model, delay=delay)

    elif REPLACEMENT_OR_REMOVAL =='REMOVAL':
        modified_text = content_remove_text(content=row['text'], model=model, delay=delay)


    return modified_text

def get_modified_text_index(text):
    return text.find('**MODIFIED_TEXT:**')

def get_text_after_modified_text(text):
    index = get_modified_text_index(text)
    return text[index + len('**MODIFIED_TEXT:**'):].strip() if index != -1 else text

def machine_summary_anon(index_row_tuple, model, delay=0, REPLACEMENT_OR_REMOVAL=None ):
    print('index_row_tuple ', index_row_tuple)
    row = index_row_tuple
    machine_summaries = row['machine_summaries']
    print('machine summaries ', machine_summaries  )

    if REPLACEMENT_OR_REMOVAL =='REPLACEMENT':
        replaced_summaries = [content_replacement_of_text(content=summary, model=model, delay=delay) for summary in machine_summaries]

    elif REPLACEMENT_OR_REMOVAL =='REMOVAL':
        replaced_summaries = [content_remove_text(content=summary, model=model, delay=delay) for summary in machine_summaries]
    return replaced_summaries


def human_summary_anon(index_row_tuple, model, delay=0, REPLACEMENT_OR_REMOVAL=None):
    row = index_row_tuple
    human_summaries = row['human_summaries']
    print('human_summaries ', human_summaries)

    if REPLACEMENT_OR_REMOVAL =='REPLACEMENT':
        replaced_summaries = [content_replacement_of_text(content=summary, model=model, delay=delay) for summary in human_summaries]

    elif REPLACEMENT_OR_REMOVAL =='REMOVAL':
        replaced_summaries = [content_remove_text(content=summary, model=model, delay=delay) for summary in human_summaries]

    return replaced_summaries



def compute_cosine_similarities_with_text(row, model_name, deloc=False):
    string_text_column = (f'embedding_text_{model_name}' if deloc == False
                          else f'embedding_text_delocalized_{model_name}')
    embedding_text = np.array(row[string_text_column])

    string_machine_summary_column = (f'embedding_machine_summaries_{model_name}' if deloc == False
                                     else f'embedding_machine_summaries_delocalized_{model_name}')

    embedding_machine_summaries = [np.array(embedding) for embedding in row[string_machine_summary_column]]

    similarities = {
        'text_and_machine_summaries': [cosine_similarity([embedding_text], [embedding])[0][0] for embedding in embedding_machine_summaries],
    }

    return similarities['text_and_machine_summaries']


#

def compute_max_cosine_sim_machine_human(row, model_name, anon=False, llm_used=None):

    string_machine_summary_column = (f'embedding_machine_summaries_{model_name}' if anon == False
                                     else f'embedding_machine_summaries_anonymized_{model_name}_llm_used_{llm_used}')


    string_human_summary_column = (f'embedding_human_summaries_{model_name}' if anon == False
                                     else f'embedding_human_summaries_anonymized_{model_name}_llm_used_{llm_used}')

    embedding_human_summaries = [np.array(embedding) for embedding in row[string_human_summary_column]]
    embedding_machine_summaries = [np.array(embedding) for embedding in row[string_machine_summary_column]]

    max_similarities = []
    for machine_summary in embedding_machine_summaries:
        similarities = [cosine_similarity([machine_summary], [human_summary])[0][0] for human_summary in embedding_human_summaries]
        max_similarities.append(max(similarities))

    return max_similarities

